# Week 5 - AB Testing

Google Analytics provides APIs for others to retrieve data in a flexible way. The flexible report generation is based on the definition of **dimensions** and **metrics**.
More information about dimensions and metrics can be found [here](https://developers.google.com/analytics/devguides/reporting/data/v1/api-schema). More information about how to generate reports can be found [here](https://developers.google.com/analytics/devguides/reporting/data/v1/basics#python_3).

# 1. Import Packages
https://developers.google.com/analytics/devguides/reporting/data/v1/rest

In [1]:
!pip3 install google.analytics.data
!pip3 install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 7.7 MB/s eta 0:00:00


In [3]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
    Filter,
    FilterExpression,
)
import os
import pandas as pd
import json
import scipy.stats as st
import numpy as np
from google.colab import userdata


# 2 Report Generation
Below is a function ```sample_run_report```. The parameter is the property id, and the dimensions, metrics, and date_ranges can be specified below.

## 2.1 Conversion Report
The report below collects collects event sessions by experimental group

In [4]:
def sample_run_report_conversion(property_id="424145747"):
    """Runs a simple report on a Google Analytics 4 property."""
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = userdata.get('GOOGLE_CREDENTIALS_PATH')
    client = BetaAnalyticsDataClient()
    request = RunReportRequest(
        property="properties/{}".format(property_id),
        dimensions=[Dimension(name="campaignName"),Dimension(name="eventName"),Dimension(name="date")], #Dimension(name="sessionCampaignName"),
        metrics=[Metric(name="sessions")],
        date_ranges=[DateRange(start_date="2025-01-01", end_date="today")],
         dimension_filter=FilterExpression(
            filter=Filter(
                field_name="campaignName",
                in_list_filter=Filter.InListFilter(
                    values=[
                        "main",
                        "experiment"
                    ]
                ),
            )
        ),
    )
    response = client.run_report(request)
    return response

## 2.2 Session Report
The report below shows all experimental sessions:

In [5]:
def sample_run_report_session(property_id="424145747"):
    """Runs a simple report on a Google Analytics 4 property."""
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = userdata.get('GOOGLE_CREDENTIALS_PATH')
    client = BetaAnalyticsDataClient()
    request = RunReportRequest(
        property="properties/{}".format(property_id),
        dimensions=[Dimension(name="customEvent:Branch"),Dimension(name="date")], #Dimension(name="sessionCampaignName"),
        metrics=[Metric(name="sessions")],
        date_ranges=[DateRange(start_date="2025-01-01", end_date="today")],
          dimension_filter=FilterExpression(
            filter=Filter(
                field_name="customEvent:Branch",
                in_list_filter=Filter.InListFilter(
                    values=[
                        "main",
                        "experiment"
                    ]
                ),
            )
        ),
    )
    response = client.run_report(request)
    return response

# 3 Data Conversion
The default output is json, which we can transform into a dataframe using the following function **response_to_df**.

In [6]:
def response_to_df(response):
    columns = []
    rows = []

    for col in response.dimension_headers:
        columns.append(col.name)
    for col in response.metric_headers:
        columns.append(col.name)

    for row_data in response.rows:
        row = []
        for val in row_data.dimension_values:
            row.append(val.value)
        for val in row_data.metric_values:
            row.append(val.value)
        rows.append(row)
    return pd.DataFrame(rows, columns=columns)


# 4 Data Collection Pipeline

## 4.1 Conversion Data
The code below shows the main pipeline of this task.

In [7]:
response=sample_run_report_conversion(property_id="424145747")
df1=response_to_df(response)
print(df1)

# change the column name from  campaignName to customEvent:Branch
df1.rename(columns={'campaignName': 'customEvent:Branch'}, inplace=True)
# Only look at submit
df1 = df1[df1['eventName'] == 'submit']

   campaignName eventName      date sessions
0          main     again  20250312        7
1          main    submit  20250320        5
2    experiment     again  20250407        4
3    experiment    submit  20250407        4
4          main     again  20250407        4
5          main    submit  20250407        4
6          main     again  20250301        3
7          main     again  20250319        3
8          main    submit  20250228        3
9          main    submit  20250301        3
10         main     again  20250227        2
11         main     again  20250228        2
12         main     again  20250303        2
13         main     again  20250313        2
14         main    submit  20250227        2
15   experiment     again  20250311        1
16   experiment     again  20250312        1
17   experiment    submit  20250307        1
18   experiment    submit  20250310        1
19   experiment    submit  20250311        1
20         main     again  20250306        1
21        

## 4.2 Session Data

In [8]:
response=sample_run_report_session(property_id="424145747")
df2=response_to_df(response)
print(df2)

   customEvent:Branch      date sessions
0                main  20250313       36
1                main  20250320       19
2                main  20250312       16
3                main  20250228        9
4                main  20250319        9
5                main  20250316        8
6                main  20250317        8
7                main  20250321        6
8                main  20250227        5
9                main  20250407        5
10         experiment  20250407        4
11               main  20250301        4
12               main  20250326        4
13               main  20250303        3
14               main  20250314        3
15               main  20250318        3
16               main  20250327        3
17               main  20250306        2
18               main  20250311        2
19               main  20250315        2
20               main  20250322        2
21               main  20250324        2
22               main  20250325        2
23              

# 5. Hypothesis Testing

## 5.1 Prepare Data
To combine the two datasets, we used a merge function:

![left-outer-join-operation.png](attachment:left-outer-join-operation.png)

In [9]:


# the merge function can combine two dataframes based on keywords. A left
merged_df = pd.merge(df2, df1, on=['date', 'customEvent:Branch'], how='left')

merged_df.fillna(0, inplace=True)

# calculate the ratio between submission sessions and all sessions per day.
merged_df['submit_session_ratio_per_day']=merged_df['sessions_y'].astype(int)/merged_df['sessions_x'].astype(int)

# extract data for control
control = merged_df[merged_df['customEvent:Branch'] == 'main']['submit_session_ratio_per_day']
# extract data for treatment
treatment =merged_df[merged_df['customEvent:Branch'] == 'experiment']['submit_session_ratio_per_day']


## 5.2 T-Test Assuming Equal Variance
This is a test for the null hypothesis that 2 independent samples have identical average (expected) values. This test assumes that the populations have identical variances by default.

In [10]:
# Calculate sample means for both groups
mean_control = np.mean(control)
mean_treatment = np.mean(treatment)

# Calculate sample standard deviations for both groups
std_control = np.std(control)
std_treatment = np.std(treatment)


t_stat, p_value = st.ttest_ind(treatment, control)

# Define significance level
alpha = 0.05

# Check if the difference is statistically significant
if p_value < alpha:
    print("The difference is statistically significant.")
else:
    print("The difference is not statistically significant.")

# Output the results
print(f"Control Group Mean: {mean_control}")
print(f"Treatment Group Mean: {mean_treatment}")
print(f"P-Value: {p_value}")

The difference is statistically significant.
Control Group Mean: 0.11654567453115547
Treatment Group Mean: 0.5
P-Value: 0.004552170169304439


In [11]:
from pingouin import ttest

ttest(treatment,control,correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.032026,35,two-sided,0.004552,"[0.13, 0.64]",1.210848,8.933,0.838402


## 5.3 T-Test Assuming Unequal Variance

In [12]:
from pingouin import ttest

ttest(treatment,control,correction=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.978199,7.742731,two-sided,0.084475,"[-0.07, 0.83]",1.210848,1.491,0.838402


# 6. Practice 1: Channel Comparison
Compare the session numbers for the two campaigns: one with medium being "announcement" and the other with medium being "canvas".

In [15]:
# Practice
def sample_run_report_practice(property_id="424145747"):
    """Runs a simple report on a Google Analytics 4 property."""
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = userdata.get('GOOGLE_CREDENTIALS_PATH')
    client = BetaAnalyticsDataClient()
    request = RunReportRequest(
        property="properties/{}".format(property_id),
        dimensions=[Dimension(name="source"),Dimension(name="medium"),Dimension(name="date")], #Dimension(name="sessionCampaignName"),
        metrics=[Metric(name="sessions")],
        date_ranges=[DateRange(start_date="2024-01-01", end_date="today")],
         dimension_filter=FilterExpression(
            filter=Filter(
                field_name="source",
                in_list_filter=Filter.InListFilter(
                    values=[
                        "social"
                    ]
                ),
            )
        ),
    )
    response = client.run_report(request)
    return response

In [14]:
response=sample_run_report_practice(property_id="424145747")
df=response_to_df(response)
print(df)

    source        medium      date sessions
0   social  announcement  20250312        3
1   social        canvas  20240311        3
2   social  announcement  20240310        2
3   social  announcement  20250311        2
4   social  announcement  20250407        2
5   social        canvas  20240309        2
6   social  announcement  20240309        1
7   social  announcement  20240311        1
8   social  announcement  20240322        1
9   social  announcement  20250306        1
10  social  announcement  20250307        1
11  social  announcement  20250310        1
12  social  announcement  20250316        1
13  social  announcement  20250320        1
14  social  announcement  20250325        1
15  social        canvas  20240322        1


In [17]:
from scipy import stats
from scipy.stats import ttest_ind

# Hypothesis testing

control = df[df['medium'] == 'canvas']['sessions'].astype(int)
# extract data for treatment
treatment =df[df['medium'] == 'announcement']['sessions'].astype(int)


> Complete the code to check whether the average session numberis different across these two channels.

# 7. Practice 2: Feedback Comparison

> On a daily level, do you get more thumbs ups than thumbs downs?